In [1]:
#conda install -c anaconda py-xgboost
# pip install xgboost
from xgboost import XGBClassifier

In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
#import ipynb.fs.full.data_preprocessing_after_imputing as datapre

## XGBoost Sampling

In [3]:
# Tunning Paramter Space
parameter_space = {
        'learning_rate':[0.1],
        'n_estimators': [100],
        'max_depth':[5],
        'min_child_weight':[1],
        'gamma':[0],
        'subsample':[0.8],
        'colsample_bytree':[0.8],
        'objective': ['binary:logistic'],
        'nthread': [4],
        'scale_pos_weight':[1],
        'seed':[27]
    }


In [4]:
def best_xgboost(datatrain, datavali, parameter_space,criteria='accuracy', randomstate=0):
    '''
    if not specified, the parameters are n_estimators=100, criterion="gini", max_depth=8, min_samples_split=400, 
    min_samples_leaf=50, bootstrap=True, oob_score=False, warm_start=False, max_samples=3000, random_state=0
    '''
    from sklearn.model_selection import PredefinedSplit
    import numpy as np
    from xgboost.sklearn import XGBClassifier
    
    frames=pd.concat([datatrain, datavali])
    frames_reind=frames.set_index([pd.Index(range(len(frames)))])
    split_index=[-1]*len(datatrain)+[0]*len(datavali)
    pds = PredefinedSplit(test_fold = split_index)
    datadummy=pd.get_dummies(frames_reind)
    datatrainX=np.array(datadummy.drop('Default_ind', axis=1))
    datatrainy=datadummy['Default_ind']
    
    XGB=XGBClassifier()
    from sklearn.model_selection import GridSearchCV
    
    modelGS=GridSearchCV(XGB, parameter_space,cv=pds,scoring=criteria)
    modelGS.fit(datatrainX, datatrainy)
    
    return modelGS,datatrainX,datatrainy

## Treat as Categorical data

In [5]:
def Cate_to_object_excp_inq(dataset):
    return(dataset.astype({'non_mtg_acc_past_due_12_months_num': 'object','non_mtg_acc_past_due_6_months_num': 'object','mortgages_past_due_6_months_num': 'object','card_inq_24_month_num': 'float64','inq_12_month_num': 'float64','card_open_36_month_num': 'object','auto_open_36_month_num': 'object','ind_acc_XYZ': 'object'}))

## Upsampling

In [6]:
def upsample(data):
    from sklearn.utils import resample
    
    df_majority=data[data.Default_ind==0]
    df_minority=data[data.Default_ind==1]
    largenumber=len(df_majority)
    df_minority_upsampled=resample(df_minority,replace=True,n_samples=int(1.2*largenumber),random_state=123)
    df_upsampled=pd.concat([pd.DataFrame(df_majority),pd.DataFrame(df_minority_upsampled)])
    return df_upsampled

## KNN data

In [7]:
knn_train=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/imputed knn/KNN_impute.csv')
knn_train=knn_train.drop(knn_train.columns[0],axis=1)
knn_test=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/imputed knn/KNN_impute_test.csv')
knn_test=knn_test.drop(knn_test.columns[0],axis=1)
knn_validation=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/imputed knn/KNN_impute_validation.csv')
knn_validation=knn_validation.drop(knn_validation.columns[0],axis=1)

In [8]:
cat_knn_train=Cate_to_object_excp_inq(knn_train)
cat_knn_test=Cate_to_object_excp_inq(knn_test)
cat_knn_validation=Cate_to_object_excp_inq(knn_validation)

### Upsample train and validation

In [9]:
cat_knn_up_train=upsample(cat_knn_train)
cat_knn_up_validation=upsample(cat_knn_validation)

In [10]:
clf1,x1,y1=best_xgboost(cat_knn_up_train,cat_knn_up_validation,parameter_space,criteria='accuracy', randomstate=0)
clf1.score(x1,y1)

0.8615001823212716

In [11]:
datadummy=pd.get_dummies(cat_knn_test)
testX=np.array(datadummy.drop('Default_ind', axis=1))
testy=datadummy['Default_ind']

In [12]:
clf1.score(testX,testy)

0.824

In [13]:
y_pred=clf1.predict(testX)
from sklearn.metrics import confusion_matrix
confusion_matrix( testy,y_pred)

array([[3825,  774],
       [ 106,  295]])

### Tunning Parameter KNN

In [14]:
parameter_space = {
    'colsample_bytree': [0.8,0.9,1.0],
     'gamma': [0,0.5,1],
     'learning_rate': [0.2,0.3],
     'max_depth': [3,4,5],
     'min_child_weight': [0.5,1],
     'n_estimators': [30,40,50],
     'nthread': [4,7,9],
     'objective': ['binary:logistic'],
     'scale_pos_weight': [1],
     'seed': [123],
     'subsample': [0.5,1]
    }

In [15]:
clf1,x1,y1=best_xgboost(cat_knn_up_train,cat_knn_up_validation,parameter_space,criteria='accuracy', randomstate=0)

In [16]:
clf1.best_params_

{'colsample_bytree': 1.0,
 'gamma': 1,
 'learning_rate': 0.3,
 'max_depth': 4,
 'min_child_weight': 1,
 'n_estimators': 30,
 'nthread': 4,
 'objective': 'binary:logistic',
 'scale_pos_weight': 1,
 'seed': 123,
 'subsample': 0.5}

In [17]:
y_pred=clf1.predict(testX)
from sklearn.metrics import confusion_matrix
confusion_matrix( testy,y_pred)

array([[3724,  875],
       [  84,  317]])

In [18]:
clf1.score(testX,testy)

0.8082

## Cart

In [31]:
#Import dataset
cart_train=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/Cart impute/Cart_impute.csv')
cart_test=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/Cart impute/Cart_impute_Test.csv')
cart_validation=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/Cart impute/Cart_impute_Vali.csv')

In [32]:
#Change to categorical
cat_cart_train=Cate_to_object_excp_inq(cart_train)
cat_cart_test=Cate_to_object_excp_inq(cart_test)
cat_cart_validation=Cate_to_object_excp_inq(cart_validation)

In [33]:
# Upsample train and validation
cat_cart_up_train=upsample(cat_cart_train)
cat_cart_up_validation=upsample(cat_cart_validation)

In [34]:
#test data Set
datadummy=pd.get_dummies(cat_cart_test)
testX=np.array(datadummy.drop('Default_ind', axis=1))
testy=datadummy['Default_ind']

### Tuning Parameter Cart

In [35]:
parameter_space = {
    'colsample_bytree': [0.8,0.9,1.0],
     'gamma': [0,0.5,1],
     'learning_rate': [0.2,0.3],
     'max_depth': [3,4,5],
     'min_child_weight': [0.5,1],
     'n_estimators': [30,40,50],
     'nthread': [4,7,9],
     'objective': ['binary:logistic'],
     'scale_pos_weight': [1],
     'seed': [123],
     'subsample': [0.5,1]
    }

In [36]:
clf2,x2,y2=best_xgboost(cat_cart_up_train,cat_cart_up_validation,parameter_space,criteria='accuracy', randomstate=0)

In [37]:
clf2.best_params_

{'colsample_bytree': 0.9,
 'gamma': 0,
 'learning_rate': 0.3,
 'max_depth': 3,
 'min_child_weight': 0.5,
 'n_estimators': 40,
 'nthread': 4,
 'objective': 'binary:logistic',
 'scale_pos_weight': 1,
 'seed': 123,
 'subsample': 0.5}

In [38]:
y_pred=clf2.predict(testX)
from sklearn.metrics import confusion_matrix
confusion_matrix(testy,y_pred)

array([[3696,  903],
       [  80,  321]])